In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
from PIL import Image
import os


In [ ]:
def resize_images(input_dir, output_dir, size=(224, 224)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file)
                try:
                    img = Image.open(file_path)
                    img = img.resize(size, Image.ANTIALIAS)

                    # Save to the corresponding location in the output directory
                    relative_path = os.path.relpath(root, input_dir)
                    output_path = os.path.join(output_dir, relative_path)

                    if not os.path.exists(output_path):
                        os.makedirs(output_path)

                    img.save(os.path.join(output_path, file))
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

In [ ]:
model = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 3s 0us/step


In [ ]:
train_dir = '/content/drive/MyDrive/dataset_resized/train'
test_dir = '/content/drive/MyDrive/dataset_resized/test'


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Modelinizin girdi boyutuna göre ayarlayın
    batch_size=32,
    class_mode='binary'  # İkili sınıflandırma için 'binary', çoklu sınıflandırma için 'categorical' kullanın
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 2216 images belonging to 5 classes.
Found 632 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

# Önceden eğitilmiş VGG16 modelini yükleyin ve transfer öğrenme için kullanın
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Yeni bir model oluşturun
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Bazı katmanları dondurun (eğitim sırasında değiştirilmesin)
for layer in base_model.layers:
    layer.trainable = False

# Modeli derleyin
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=3
)


Epoch 1/3
69/69 [==============================] - 785s 11s/step - loss: -1543.8145 - accuracy: 0.1255 - val_loss: -3461.4368 - val_accuracy: 0.1053
Epoch 2/3
69/69 [==============================] - 40s 580ms/step - loss: -7965.3242 - accuracy: 0.1227 - val_loss: -11674.4189 - val_accuracy: 0.1069
Epoch 3/3
69/69 [==============================] - 40s 577ms/step - loss: -20683.0156 - accuracy: 0.1223 - val_loss: -25417.8906 - val_accuracy: 0.1053


In [ ]:
model.save('/content/drive/MyDrive/dataset_resized/model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def load_and_prep_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_image(model, image_path):
    img_array = load_and_prep_image(image_path)
    prediction = model.predict(img_array)
    return prediction

# Örnek görüntü dosyasının yolunu belirleyin
image_path = '/content/drive/MyDrive/dataset_resized/test görüntüleri/2024-rolls-royce-phantom-102-64bad70ba7661.jpg'
prediction = predict_image(model, image_path)
print(f'Prediction: {prediction}')


1/1 [==============================] - 1s 1s/step
Prediction: [[1.]]


In [ ]:
class_names = ['Audi', 'Hyundai Creta', 'Swift', 'Rolls Royce', 'Toyota Innova']


In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/dataset_resized/model.h5')

In [ ]:
img_path = '/content/87c5d29d2bd3b2dda371427fe9427b60.jpg'  # Test etmek istediğiniz resmin yolu
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Aynı ölçeklendirme işlemini uygulayın

In [ ]:
prediction = model.predict(img_array)

def decode_prediction(prediction, class_names):
    predicted_class = class_names[np.argmax(prediction[0])]
    return predicted_class

predicted_class = decode_prediction(prediction, class_names)
print(f'Predicted Class: {predicted_class}')


1/1 [==============================] - 0s 20ms/step
Predicted Class: Audi


In [ ]:
prediction = model.predict(img_array)

def decode_prediction(prediction, class_names):
    predicted_class = class_names[np.argmax(prediction[0])]
    return predicted_class

predicted_class = decode_prediction(prediction, class_names)
print(f'Predicted Class: {predicted_class}')

1/1 [==============================] - 0s 18ms/step
Predicted Class: Audi


In [ ]:
import os
from collections import Counter

def count_files(directory):
    counter = Counter()
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('png', 'jpg', 'jpeg')):
                counter[os.path.basename(root)] += 1
    return counter

train_counter = count_files(train_dir)
test_counter = count_files(test_dir)

print(f'Train data distribution: {train_counter}')
print(f'Test data distribution: {test_counter}')


Train data distribution: Counter({'Toyota Innova': 670, 'Audi': 540, 'Swift': 424, 'Rolls Royce': 311, 'Hyundai Creta': 271})
Test data distribution: Counter({'Audi': 199, 'Toyota Innova': 190, 'Swift': 102, 'Rolls Royce': 74, 'Hyundai Creta': 67})


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Modeli yükleyin
model = tf.keras.models.load_model('/content/drive/MyDrive/dataset_resized/model.h5')

# Sınıf isimlerini belirleyin
class_names = ['Hyundai Creta','Audi',  'Swift', 'Rolls Royce', 'Toyota Innova']

# Test etmek istediğiniz resmi yükleyin ve ön işleyin
img_path = '/content/drive/MyDrive/dataset_resized/test/Rolls Royce/159.jpg'  # Test etmek istediğiniz resmin yolu
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Aynı ölçeklendirme işlemini uygulayın

# Tahmini alın
prediction = model.predict(img_array)

# Tahmini decode edin
def decode_prediction(prediction, class_names):
    predicted_class = class_names[np.argmax(prediction[0])]
    return predicted_class

predicted_class = decode_prediction(prediction, class_names)
print(f'Predicted Class: {predicted_class}')


1/1 [==============================] - 0s 209ms/step
Predicted Class: Hyundai Creta


In [ ]:
# Gerekli kütüphaneleri yükleyin
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
from tensorflow.keras.preprocessing import image
from sklearn.utils.class_weight import compute_class_weight

# Google Drive'ı bağlayın
from google.colab import drive
drive.mount('/content/drive')

# Veri yollarını belirleyin
train_dir = '/content/drive/MyDrive/dataset_resized/train'
test_dir = '/content/drive/MyDrive/dataset_resized/test'

# Veri artırma ve yükleme
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Multiple classes
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Multiple classes
)

# Modeli oluşturma
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)  # Output layer for multiple classes

model = Model(inputs=base_model.input, outputs=predictions)

# Bazı katmanları dondurun
for layer in base_model.layers:
    layer.trainable = False

# Modeli derleyin
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Sınıf ağırlıklarını hesapla
class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = dict(enumerate(class_weights))

# Modeli eğitin
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    class_weight=class_weights
)

# Modeli kaydetme
model.save('/content/drive/MyDrive/dataset_resized/model.h5')

# Sınıf isimlerini belirleyin
class_names = ['Audi', 'Hyundai Creta', 'Swift', 'Rolls Royce', 'Toyota Innova']

# Modeli yükleyin
model = tf.keras.models.load_model('/content/drive/MyDrive/dataset_resized/model.h5')

# Test etmek istediğiniz resmi yükleyin ve ön işleyin
img_path = '/content/drive/MyDrive/dataset_resized/test/Rolls Royce/176.jpg'  # Test etmek istediğiniz resmin yolu
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Aynı ölçeklendirme işlemini uygulayın

# Tahmini alın
prediction = model.predict(img_array)

# Tahmini decode edin
def decode_prediction(prediction, class_names):
    predicted_class = class_names[np.argmax(prediction[0])]
    return predicted_class

predicted_class = decode_prediction(prediction, class_names)
print(f'Predicted Class: {predicted_class}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 2216 images belonging to 5 classes.
Found 632 images belonging to 5 classes.
Epoch 1/10
69/69 [==============================] - 42s 584ms/step - loss: 1.7058 - accuracy: 0.5119 - val_loss: 0.8097 - val_accuracy: 0.7204
Epoch 2/10
69/69 [==============================] - 40s 574ms/step - loss: 0.5962 - accuracy: 0.8255 - val_loss: 0.5505 - val_accuracy: 0.8125
Epoch 3/10
69/69 [==============================] - 40s 586ms/step - loss: 0.4064 - accuracy: 0.8796 - val_loss: 0.3466 - val_accuracy: 0.9046
Epoch 4/10
69/69 [==============================] - 39s 570ms/step - loss: 0.2645 - accuracy: 0.9318 - val_loss: 0.3086 - val_accuracy: 0.9013
Epoch 5/10
69/69 [==============================] - 38s 548ms/step - loss: 0.2089 - accuracy: 0.9515 - val_loss: 0.3587 - val_accuracy: 0.8882
Epoch 6/10
69/69 [==============================] - 40s 568ms/step - loss

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 145ms/step
Predicted Class: Swift


In [ ]:
# Test etmek istediğiniz resmi yükleyin ve ön işleyin
img_path = '/content/drive/MyDrive/dataset_resized/test görüntüleri/800px-2019_Rolls-Royce_Cullinan_V12_Automatic_6.75_Front.jpg'  # Test etmek istediğiniz resmin yolu
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Aynı ölçeklendirme işlemini uygulayın

# Tahmini alın
prediction = model.predict(img_array)

# Tahmini decode edin
def decode_prediction(prediction, class_names):
    predicted_class = class_names[np.argmax(prediction[0])]
    return predicted_class

predicted_class = decode_prediction(prediction, class_names)
print(f'Predicted Class: {predicted_class}')


1/1 [==============================] - 0s 31ms/step
Predicted Class: Swift


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet, preprocess_input, decode_predictions
import numpy as np

# MobileNet modelini yükleyin
model = MobileNet(weights='imagenet')

# Test etmek istediğiniz resmin yolu
img_path = '/content/drive/MyDrive/dataset_resized/test görüntüleri/800px-2019_Rolls-Royce_Cullinan_V12_Automatic_6.75_Front.jpg'

# Resmi yükleyin ve ön işleyin
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Tahmini alın
preds = model.predict(x)

# Tahmini yazdırın
print('Predicted:', decode_predictions(preds, top=3)[0])


ImportError: cannot import name 'preprocess_input' from 'tensorflow.keras.applications' (/usr/local/lib/python3.10/dist-packages/keras/api/_v2/keras/applications/__init__.py)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
import numpy as np

# MobileNet modelini yükleyin
model = MobileNet(weights='imagenet')

# Test etmek istediğiniz resmin yolu
img_path = '/content/drive/MyDrive/görüntü işleme/test/Rolls Royce/115.jpg'

# Resmi yükleyin ve ön işleyin
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Tahmini alın
preds = model.predict(x)

# Tahmini yazdırın
print('Predicted:', decode_predictions(preds, top=3)[0])


35363/35363 [==============================] - 0s 0us/step
Predicted: [('n02814533', 'beach_wagon', 0.2681671), ('n02974003', 'car_wheel', 0.2628276), ('n04285008', 'sports_car', 0.22201599)]
